# Using transformers for SeriesBasedCesnetDataset

### Import

In [1]:
import numpy as np
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TransformerType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import SeriesBasedConfig # Series based dataset MUST use SeriesBasedConfig

from cesnet_tszoo.utils.transformer import Transformer # For creating custom Transformer

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
series_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.IP_ADDRESSES_SAMPLE, aggregation=AgreggationType.AGG_10_MINUTES, is_series_based=True, display_details=True)

[2025-08-05 19:45:22,600][wrapper_dataset][INFO] - Dataset is series-based. Use cesnet_tszoo.configs.SeriesBasedConfig



Dataset details:

    AgreggationType.AGG_10_MINUTES
        Time indices: range(0, 40297)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 3, 49, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 50, 52, tzinfo=datetime.timezone.utc))

    SourceType.IP_ADDRESSES_SAMPLE
        Time series indices: [ 11  20 101 103 118 ... 2003134 2008461 2011839 2022235 2044888], Length=1000; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'n_dest_ip': 0, 'n_dest_asn': 0, 'n_dest_ports': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Transformers

- Transformers are implemented as class.
    - You can create your own or use built-in one.
- Transformer is applied after `default_values` and fillers took care of missing values.
- One transformer is used for all time series.
- Transformer must implement `transform`.
- Transformer must implement `partial_fit` (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- To use transformer, train set must be implemented (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- You can change used transformer later with `update_dataset_config_and_initialize` or `apply_transformer`.

#### Built-in

In [4]:
# Options

## Supported
TransformerType.STANDARD_SCALER
TransformerType.L2_NORMALIZER
TransformerType.LOG_TRANSFORMER
TransformerType.MAX_ABS_SCALER
TransformerType.MIN_MAX_SCALER

<TransformerType.MIN_MAX_SCALER: 'min_max_scaler'>

In [5]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-05 19:45:22,623][config][INFO] - Quick validation succeeded.
[2025-08-05 19:45:22,740][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:22,745][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1107.98it/s]
[2025-08-05 19:45:23,208][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:23,209][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Batch sizes
        Train batch size: 32
        Val batch size: 64
 

In [6]:
series_based_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [7]:
series_based_dataset.get_transformers()

Or later with:

In [8]:
series_based_dataset.update_dataset_config_and_initialize(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)
# Or
series_based_dataset.apply_transformer(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)

[2025-08-05 19:45:23,448][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-05 19:45:23,476][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:23,479][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 367.02it/s]
[2025-08-05 19:45:23,643][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:23,644][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-05 19:45:23,645][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-05 19:45:23,645][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-05 19:45:23,675][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:23,678][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 397.41it/s]
[2025-08-05 19:45:23,831][cesnet_dataset]

#### Custom

You can create your own custom transformer. It is recommended to derive from Transformer base class.

In [9]:
class CustomTransformer(Transformer):
    def __init__(self):
        super().__init__()
        
        self.max = None
        self.min = None
    
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    
    def fit(self, data):
        self.partial_fit(data)
    
    def partial_fit(self, data):
        
        if self.max is None and self.min is None:
            self.max = np.max(data, axis=0)
            self.min = np.min(data, axis=0)
            return
        
        temp_max = np.max(data, axis=0)
        temp = np.vstack((self.max, temp_max)) 
        self.max = np.max(temp, axis=0)
        
        temp_min = np.min(data, axis=0)
        temp = np.vstack((self.min, temp_min)) 
        self.min = np.min(temp, axis=0)            

In [10]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-05 19:45:23,869][config][INFO] - Quick validation succeeded.
[2025-08-05 19:45:23,946][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:23,950][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1330.27it/s]
[2025-08-05 19:45:24,327][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:24,327][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Batch sizes
        Train batch size: 32
        Val batc

In [11]:
series_based_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [12]:
series_based_dataset.get_transformers()

Or later with:

In [13]:
series_based_dataset.update_dataset_config_and_initialize(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)
# Or
series_based_dataset.apply_transformer(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)

[2025-08-05 19:45:24,523][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-05 19:45:24,550][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:24,553][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 439.51it/s]
[2025-08-05 19:45:24,691][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:24,691][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-05 19:45:24,692][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-05 19:45:24,693][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-05 19:45:24,771][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:24,774][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 448.25it/s]
[2025-08-05 19:45:24,910][cesnet_dataset]

#### Using already fitted transformer

- When `partial_fit_initialized_transformer` is False (default value), transformer has no requirement for `partial_fit` nor for train set.

In [14]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=False, workers=0)

fitted_transformer = series_based_dataset.get_transformers()

[2025-08-05 19:45:24,926][config][INFO] - Quick validation succeeded.
[2025-08-05 19:45:24,955][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:24,959][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1557.60it/s]
[2025-08-05 19:45:25,283][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:25,283][cesnet_dataset][INFO] - Config initialized successfully.


In [15]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-05 19:45:25,298][config][INFO] - Quick validation succeeded.
[2025-08-05 19:45:25,327][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:25,331][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 1000/1000 [00:00<00:00, 1695.02it/s]
[2025-08-05 19:45:25,923][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:25,924][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDS: [792059 756220     20 143746 171452 ...  11069 338309  10197  41990  44252], Length=70
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ...  11069 338309  10197  41990  44252], Length=135
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers p

In [16]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,792059.0,0.0,0.000000,0.000000e+00
1,792059.0,1.0,0.000068,2.560557e-07
2,792059.0,2.0,0.000023,1.280278e-07
3,792059.0,3.0,0.000000,0.000000e+00
4,792059.0,4.0,0.000056,2.560557e-07
5,792059.0,5.0,0.000023,6.401392e-08
6,792059.0,6.0,0.000045,2.880626e-07
7,792059.0,7.0,0.000000,0.000000e+00
8,792059.0,8.0,0.000079,2.240487e-07
9,792059.0,9.0,0.000068,2.240487e-07


Below you can see how transformer works even without train set.

In [17]:
config = SeriesBasedConfig(time_period=0.5, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-05 19:45:26,138][config][INFO] - Quick validation succeeded.
[2025-08-05 19:45:26,211][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:26,214][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1677.95it/s]
[2025-08-05 19:45:26,514][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:26,515][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: None
        Val time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers partial_fitted: False
    Batch sizes
        Train batch size: 32
        Val batch

In [18]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,4380.0,0.0,0.001253,0.000005
1,4380.0,1.0,0.001162,0.000004
2,4380.0,2.0,0.000756,0.000003
3,4380.0,3.0,0.000959,0.000004
4,4380.0,4.0,0.001219,0.000005
5,4380.0,5.0,0.001275,0.000005
6,4380.0,6.0,0.001456,0.000006
7,4380.0,7.0,0.001196,0.000004
8,4380.0,8.0,0.001433,0.000006
9,4380.0,9.0,0.001072,0.000004


##### Partial fitting on train set

Makes already fitted transformer to be fitted on new train set too. Must implement `partial_fit`.

In [19]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, partial_fit_initialized_transformer=True, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-05 19:45:26,715][config][INFO] - Quick validation succeeded.
[2025-08-05 19:45:26,744][config][INFO] - Finalization and validation completed successfully.
[2025-08-05 19:45:26,748][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 1000/1000 [00:00<00:00, 1601.54it/s]
[2025-08-05 19:45:27,375][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-08-05 19:45:27,375][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDS: [406306 599190  10196 470862 368641 ... 360850  97079  11254 210412    122], Length=70
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ... 360850  97079  11254 210412    122], Length=135
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers p

In [20]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,406306.0,0.0,0.000045,1.280278e-07
1,406306.0,1.0,0.000056,1.920417e-07
2,406306.0,2.0,0.000023,6.401392e-08
3,406306.0,3.0,0.000045,1.920417e-07
4,406306.0,4.0,0.000079,2.240487e-07
5,406306.0,5.0,0.000056,1.920417e-07
6,406306.0,6.0,0.000068,2.240487e-07
7,406306.0,7.0,0.000023,6.401392e-08
8,406306.0,8.0,0.000056,1.600348e-07
9,406306.0,9.0,0.000079,2.240487e-07
